In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices()
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[1], True)

2022-04-19 20:44:21.938952: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-04-19 20:44:23.519158: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-19 20:44:23.522730: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-19 20:44:23.583857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 20:44:23.584463: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 119.24GiB/s
2022-04-19 20:44:23.584520: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-04-19 20:44:23.613476: I tensorflow/stream_executor/platform/def

In [2]:
from tensorflow.keras.layers import Dense, Input, Lambda, Flatten, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from collections import deque
import os
import gym
import numpy as np
import wandb
import random
import math
from tensorflow.keras.losses import KLDivergence
import time
from tensorflow.keras.layers import Layer

In [3]:
import datetime
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# rm -rf ./logs/train_PPO/
train_log_dir = "logs/train_PPO/" + current_time
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
%load_ext tensorboard
%tensorboard --logdir {train_log_dir}

2022-04-19 20:44:24.270929: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-19 20:44:24.271371: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 20:44:24.271516: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 119.24GiB/s
2022-04-19 20:44:24.271540: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libc

In [4]:
env_name = "BipedalWalker-v3"
total_timesteps = 7_50_000
init_lr = 4e-4
seed = 1
num_envs = 15
num_steps = 448
batch_size = num_envs * num_steps
anneal_lr = True
gamma = 0.99
lamda = 0.95
gae = True
num_minibatch = 8
minibatch_size = 192#batch_size//num_minibatch
update_epochs = 20
norm_adv = True
clip_coeff = 0.2
clip_value_loss = True
entropy_coeff = 0.01
valueloss_coeff = 0.5
max_grad_norm = 0.5
target_kl = None

In [5]:
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [6]:
env = gym.make(env_name)
env = gym.wrappers.RecordEpisodeStatistics(env)
# env = gym.wrappers.RecordVideo(env, "videos", step_trigger=lambda t: t%100==0)
env.seed(seed)


input_size = env.observation_space.shape[0]
output_size = env.action_space.shape[0]

In [7]:
input_size, output_size

(24, 4)

In [8]:
print(f"Observations Shape: {env.observation_space.shape}")
print(f"Action Shape: {env.action_space.shape}")

Observations Shape: (24,)
Action Shape: (4,)


In [9]:
output_size

4

In [10]:
class Agent:
    def __init__(self):
        self.actor = self.build_actor()
        self.critic = self.build_critic()
        self.log_std_layer = tf.Variable(initial_value = tf.keras.initializers.Zeros()(shape = output_size), trainable = True)
        
        
    def build_critic(self):
        inputs = Input(shape = input_size)
        kernel_init = tf.keras.initializers.Orthogonal(gain=np.sqrt(2.0))
        x = Dense(256, activation = 'tanh', kernel_initializer = kernel_init)(inputs)
        x = Dense(128, activation = 'tanh', kernel_initializer = kernel_init)(x)
        x = Dense(64, activation = 'tanh', kernel_initializer = kernel_init)(x)
        kernel_init = tf.keras.initializers.Orthogonal(gain=1.0)
        x = Dense(1, activation = 'linear', kernel_initializer = kernel_init)(x)
        model = Model(inputs = inputs, outputs = x)
        return model
    
    def build_actor(self):
        inputs = Input(shape = input_size)
        kernel_init = tf.keras.initializers.Orthogonal(gain=np.sqrt(2.0))
        x = Dense(256, activation = 'tanh', kernel_initializer = kernel_init)(inputs)
        x = Dense(128, activation = 'tanh', kernel_initializer = kernel_init)(x)
        x = Dense(64, activation = 'tanh', kernel_initializer = kernel_init)(x)
        kernel_init = tf.keras.initializers.Orthogonal(gain=0.01)
        mean = Dense(output_size , activation = 'linear', kernel_initializer = kernel_init)(x)
        model = Model(inputs = inputs, outputs = mean)
        return model
    
    def save(self):
        self.actor.save_weights("actor.hdf5")
        self.critic.save_weights("critic.hdf5")
        with open('log_std.hdf5', 'wb') as f:
            np.save(f, agent.log_std_layer)
    
    
    def load(self):
        self.actor.load_weights("actor.hdf5")
        self.critic.load_weights("critic.hdf5")
        with open('log_std.hdf5', 'rb') as f:
            agent.log_std_layer = tf.Variable(np.load(f, allow_pickle=True))
    
    def get_value(self, x):
        return self.critic(x)
    
    @tf.function
    def get_actor_critic_value(self, x, action = None):
        mean = tf.nn.tanh(self.actor(x))
        log_std = tf.expand_dims(self.log_std_layer, axis = 0)
        vector_size = mean.shape[0]
        log_std  = tf.repeat(log_std, repeats = vector_size, axis =0)
        cov_diag = tf.math.exp(log_std)
        dis = tfp.distributions.MultivariateNormalDiag(loc=mean,
                                                       scale_diag=cov_diag,
                                                       validate_args=True,
                                                       allow_nan_stats=False)
        if action is None:
            action = tf.clip_by_value(dis.sample(), -1, 1) 
        return tf.squeeze(action), dis.log_prob(action), dis.entropy(), tf.squeeze(self.critic(x))
    
    @tf.function
    def optimization(self, st, ac, log_pr, val, ret, adv):
        with tf.GradientTape(persistent = True) as tape:
            _, new_log_prob, new_entropy, new_value = agent.get_actor_critic_value(st, ac)
            log_ratio = new_log_prob - log_pr
            ratio = tf.math.exp(log_ratio)
                
                # debugging metrics
            approx_kl = tf.reduce_mean((ratio - 1) - log_ratio)
            clip_fracs_elem = [tf.reduce_mean(tf.cast(tf.abs(ratio - 1.0) > clip_coeff, dtype = tf.float32))]
                
            clipped_ratio = tf.clip_by_value(ratio, 1-clip_coeff, 1+clip_coeff)
                
            mb_advantages = adv

            if norm_adv:
                mb_advantages = (mb_advantages - tf.math.reduce_mean(mb_advantages))/(tf.math.reduce_std(mb_advantages) + 1e-8)

            ppo_loss = tf.reduce_mean(tf.math.maximum(-ratio * mb_advantages, -clipped_ratio * mb_advantages))

            if clip_value_loss:
                v_loss_unclipped = (new_value - ret) ** 2
                v_clipped = val + tf.clip_by_value(new_value - val, -clip_coeff, clip_coeff)
                v_loss_clipped = (v_clipped - ret) ** 2
                v_loss = 0.5 * tf.reduce_mean(tf.math.maximum(v_loss_unclipped, v_loss_clipped))
            else:
                v_loss = 0.5 * tf.reduce_mean((new_value - ret) ** 2)
                

            entropy_loss = tf.reduce_mean(new_entropy)
            loss = ppo_loss + valueloss_coeff * v_loss - entropy_coeff * entropy_loss
            
        actor_grads = tape.gradient(loss, agent.actor.trainable_weights)
        std_grads = tape.gradient(loss, agent.log_std_layer)
        critic_grads = tape.gradient(loss, agent.critic.trainable_weights)
            
        actor_grads = [tf.clip_by_norm(g, max_grad_norm) for g in actor_grads]
        critic_grads = [tf.clip_by_norm(g, max_grad_norm) for g in critic_grads]
        std_grads = [tf.clip_by_norm(std_grads, max_grad_norm)]
            
        optimizer.apply_gradients(zip(actor_grads, agent.actor.trainable_weights))
        optimizer.apply_gradients(zip(critic_grads, agent.critic.trainable_weights))
        optimizer.apply_gradients(zip(std_grads, [agent.log_std_layer]))
        
        return approx_kl, clip_fracs_elem, clipped_ratio, ppo_loss, v_loss, entropy_loss
    


In [11]:
agent = Agent()

In [12]:
agent.load()
# agent.log_std_layer = tf.Variable(initial_value = tf.keras.initializers.Zeros()(shape = output_size), trainable = True)

In [ ]:
#Training

states = np.zeros(shape = (num_steps, num_envs) + env.observation_space.shape, dtype = np.float32)
actions = np.zeros(shape = ((num_steps, num_envs)+ env.action_space.shape), dtype = np.float32)
log_probs = np.zeros(shape = (num_steps, num_envs), dtype = np.float32)
rewards = np.zeros(shape = (num_steps, num_envs), dtype = np.float32)
dones = np.zeros(shape = (num_steps, num_envs), dtype = np.float32)
values = np.zeros(shape = (num_steps, num_envs), dtype = np.float32)


global_step = 0
start_time = time.time()
next_states = np.array([env.reset() for _ in range(num_envs)])
next_dones = np.zeros(num_envs)
num_updates = total_timesteps//batch_size
optimizer = Adam(epsilon = 1e-5)

for update in range(1, num_updates + 1):
    if anneal_lr:
        frac = 1.0 - (update - 1.0)/num_updates
        lr_now = frac * init_lr
        optimizer.lr = lr_now
    
    for e in range(num_envs):

        
        next_state = np.array(env.reset())
        done = False
        for step in range(num_steps):
            global_step += 1 
            states[step, e] = next_states[e]
            dones[step, e] =  next_dones[e]
            action, log_prob, _, value = agent.get_actor_critic_value(np.expand_dims(next_state, axis=0))
            actions[step, e] = action.numpy()
            log_probs[step, e] = log_prob
            values[step, e] = value
            
            next_state, reward, done, info = env.step(action)
            rewards[step, e] = reward

            next_states[e] = next_state
            next_dones[e] = done
            
            if "episode" in info.keys():
                episodic_return = info["episode"]["r"]
                episode_length = info["episode"]["l"]
                print(f"Global Step={global_step}, Episode Length={episode_length} Episodic_return={episodic_return}")
                with train_summary_writer.as_default():
                    tf.summary.scalar("Charts/episodic_return", episodic_return, global_step)
                    tf.summary.scalar("Charts/episodic_length", episode_length, global_step)
            
            
        
            if done and step < num_steps-1:
                next_state = np.array(env.reset())
                done = False
        
        
        
    next_value = np.squeeze(agent.get_value(next_states))
    if gae:
        advantages = np.zeros_like(rewards)
        last_gae = 0.0
        for step in reversed(range(num_steps)):
            if step == num_steps - 1:
                next_non_terminal = 1.0 - next_dones
                next_values = next_value
            else:
                next_non_terminal = 1.0 - dones[step+1]
                next_values = values[step+1]
                
            delta = rewards[step] + gamma * next_values * next_non_terminal - values[step]     
            advantages[step] = last_gae = delta + gamma * lamda * next_non_terminal * last_gae
        returns = advantages + values
    else:
        returns = np.zeros_like(rewards)
        for step in reversed(range(num_steps)):
            if step == num_steps -1:
                next_non_terminal  = 1.0 - next_dones
                next_return = next_value
            else:
                next_non_terminal = 1.0 - dones[step+1]
                next_return = returns[step+1]
            returns[step] = rewards[step] + gamma * next_non_terminal * next_return
        advantages = returns - values
        
    b_states = states.reshape((-1,)+ env.observation_space.shape)
    b_actions = actions.reshape((-1,)+ env.action_space.shape)
    b_log_probs = log_probs.reshape((-1,))
    b_values = values.reshape((-1,))
    b_advantages = advantages.reshape((-1,))
    b_returns = returns.reshape((-1,))
    
    b_indx = np.arange(batch_size)
    clip_fracs = []
    for epoch in range(update_epochs):
        np.random.shuffle(b_indx)
        for start in range(0, batch_size, minibatch_size):
            end = start + minibatch_size
            mb_indx = b_indx[start:end]
            approx_kl, clip_fracs_elem, clipped_ratio, ppo_loss, v_loss, entropy_loss = agent.optimization(b_states[mb_indx], b_actions[mb_indx], b_log_probs[mb_indx], b_values[mb_indx], b_returns[mb_indx], b_advantages[mb_indx])
            clip_fracs += clip_fracs_elem   
        if target_kl is not None:
            if approx_kl > target_kl:
                break
        
    var_true = np.var(b_returns)
    explained_var = np.nan if var_true == 0 else 1 - np.var(b_returns - b_values)/var_true
    with train_summary_writer.as_default():
        tf.summary.scalar("Charts/learning_rate", optimizer.lr, global_step)
        tf.summary.scalar("losses/value_loss", v_loss, global_step)
        tf.summary.scalar("losses/policy_loss", ppo_loss, global_step)
        tf.summary.scalar("losses/entropy", entropy_loss, global_step)
        tf.summary.scalar("losses/approx_kl", approx_kl, global_step)
        tf.summary.scalar("losses/clipfrac", np.mean(clip_fracs), global_step)
        tf.summary.scalar("losses/explained_variance", explained_var, global_step)
        print("SPS:", int(global_step / (time.time() - start_time)))
        tf.summary.scalar("Charts/SPS", int(global_step / (time.time() - start_time)), global_step)
env.close()
# train_summary_writer.close()

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it with `scale_diag` directly instead.
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.


2022-04-19 17:23:47.296867: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-04-19 17:23:47.392438: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-19 17:23:47.411200: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400000000 Hz
2022-04-19 17:23:47.455201: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


Global Step=169, Episode Length=169 Episodic_return=-93.22462463378906
Global Step=250, Episode Length=81 Episodic_return=-120.25930786132812
Global Step=2069, Episode Length=277 Episodic_return=-122.99919128417969
Global Step=2372, Episode Length=132 Episodic_return=-115.88034057617188
Global Step=2895, Episode Length=207 Episodic_return=-95.44608306884766
Global Step=3073, Episode Length=178 Episodic_return=-93.63600158691406
Global Step=3321, Episode Length=185 Episodic_return=-94.33240509033203
Global Step=3566, Episode Length=245 Episodic_return=-97.7161636352539
Global Step=3748, Episode Length=164 Episodic_return=-92.5925521850586
Global Step=4325, Episode Length=293 Episodic_return=-101.45263671875
Global Step=5448, Episode Length=72 Episodic_return=-121.29289245605469
Global Step=6015, Episode Length=191 Episodic_return=-93.62310791015625
SPS: 267
Global Step=7803, Episode Length=187 Episodic_return=-94.71603393554688
Global Step=8923, Episode Length=411 Episodic_return=-120.9

In [13]:
#Testing

next_state = env.reset()
done = False
while not done:
    env.render()
    action, log_prob, entropy, _ = agent.get_actor_critic_value(np.expand_dims(next_state, axis=0))
    next_state, reward, done, info = env.step(action)
    if "episode" in info.keys():
        episodic_return = info["episode"]["r"]
        l = info["episode"]["l"]
        print(f"Episodic_return={episodic_return}, Episode Length: {l}")
env.close()   

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it with `scale_diag` directly instead.
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.


2022-04-19 20:44:35.931000: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-04-19 20:44:36.031333: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-19 20:44:36.054997: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400000000 Hz
2022-04-19 20:44:36.110811: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


Episodic_return=269.1633605957031, Episode Length: 1416


In [ ]:
agent.save()